In [1]:
#imports needed to scrape data and transform 
import time 

#imports needed to sign into facebook 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys 
from getpass import getpass
from selenium.webdriver.common.action_chains import ActionChains

import keyring
import pyperclip
import pandas as pd 






In [58]:
def Saving_Cred():
    username = input('Enter Your Facebook Username: ')
    password = getpass('Enter your Facebook password: ')
    
    if ( not username or not password ):
        print("Please input Your Username and Password")
        return False
        
    keyring.set_password("Facebook","username", username)
    keyring.set_password("Facebook","password",password) 
    
    return True


def login(driver):
    
    #Get passwords saved from keyring
    username = keyring.get_password("Facebook", "username")
    password = keyring.get_password("Facebook","password")
    
    txtUsername = driver.find_element('id', 'email')  
    txtUsername.send_keys(username)
    
    txtPasswd = driver.find_element('id', 'pass')
    txtPasswd.send_keys(password)
   
    login_button = driver.find_element('name', 'login')
    login_button.click()
    

def clearbox(searchbox):
    searchbox.send_keys(Keys.CONTROL + "a")  # Select all text
    searchbox.send_keys(Keys.BACKSPACE) 
    
    
def search(group,driver):
    searchbox = driver.find_element(By.XPATH,'//input[@role="combobox"]') #element of searcbox 
    searchbox.click() 
    clearbox(searchbox)
    searchbox.send_keys(group)
    searchbox.send_keys(Keys.RETURN)
    
def scroll_page(driver):
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.END)
    time.sleep(2)

def post_link(post,actions,driver): 
    
    share_class_name ="x9f619 x1n2onr6 x1ja2u2z x78zum5 x1r8uery x1iyjqo2 xs83m0k xeuugli xl56j7k x6s0dn4 xozqiw3 x1q0g3np xn6708d x1ye3gou xexx8yu xcud41i x139jcc6 x4cne27 xifccgj xn3w4p2 xuxw1ft"
    xpath_share = f".//div[@class='{share_class_name}']"
    
    sharelink = post.find_elements(By.XPATH,xpath_share)     
    actions.move_to_element(sharelink[2]).perform()
    sharelink[2].click()
    
    time.sleep(2)
    
    copy_links_class="x6s0dn4 x1q0q8m5 x1qhh985 xu3j5b3 xcfux6l x26u7qi xm0m39n x13fuv20 x972fbf x9f619 x78zum5 x1q0g3np x1iyjqo2 xs83m0k x1qughib xat24cr x11i5rnm x1mh8g0r xdj266r x2lwn1j xeuugli x18d9i69 x1sxyh0 xurb0ha xexx8yu x1n2onr6 x1ja2u2z"
    link = driver.find_elements(By.CSS_SELECTOR, "." + ".".join(copy_links_class.split()))
    
    link[6].click()
    
    copied_text = pyperclip.paste()
    return copied_text

    
    
    

In [71]:

#disables nonifications 
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")


#Obtain all user inputed information 
group = input("What housing group would you like to scrap? (Be sure to write the exact name)")
sublet = input("Fall, Winter, or Summer start date?")


#creating an instance webdriver, and loading facebook page
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.facebook.com")
time.sleep(2)


#Initial Login and Repreated Login Procress -> Using Saved Passwords 
if (keyring.get_password("Facebook", "username") and keyring.get_password("Facebook", "password")): 
    print("Credentials Exist, attempting to login into Facebook....")
    login(driver)
else: #No password is Saved 
    while True:
        print("Creditials are not saved, please input your username and password for Facebook")
        if (Saving_Cred()): 
            print("Credentials Aceppted, loggining into Facebook....")
            break
        else: 
            print("Creditials are not saved, please input your username and password for Facebook")
            
    login(driver)
    

#login unsucessful: 
try:
    error = driver.find_element(By.CLASS_NAME, "_9ay7")
    print("password incorrect")
    keyring.delete_password("Facebook", "username")
    keyring.delete_password("Facebook","password")
    driver.close()
except  NoSuchElementException:
    print("Password Accepted")
    


Credentials Aceppted, loggining into Facebook....
password incorrect


TypeError: set_password() missing 1 required positional argument: 'password'

In [60]:
#Step 2: Search for Specific Groups
search = driver.find_element(By.XPATH,'//input[@role="combobox"]')
search.click()
search.send_keys(Keys.CONTROL + "a")  # Select all text to delete 
search.send_keys(Keys.BACKSPACE) 
search.send_keys(group)
time.sleep(1)
search.send_keys(Keys.DOWN)
search.send_keys(Keys.RETURN)
time.sleep(2)

#Step 3: Move to Housing Group & Place sorting in chonrological Order  
new_url = driver.current_url + "/?sorting_setting=CHRONOLOGICAL_LISTINGS"
driver.get(new_url)
    

In [1]:

#Scroll down to obtain atleast 16 posts for each scrape 
class_name ="x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z"
xpath = f"//div[@class='{class_name}']"

all_posts = []
while(len(all_posts) < 16): 
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    all_posts = driver.find_elements(By.XPATH, xpath) 
    time.sleep(2)


NameError: name 'driver' is not defined

In [64]:

actions = ActionChains(driver)     

#Collect all the posts in the boswer: 
class_name ="x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z"
xpath = f"//div[@class='{class_name}']"
all_posts = driver.find_elements(By.XPATH, xpath)

#Checking if posts were gathered
print(len(all_posts))



#Possible indicators in text that refer to each starting term 
fall = ["fall", "september","sept", "sep"]
summer = ["summer", "may"]
winter = ["winter", "january","jan"]

#gathering information on what sublet they want 
if (sublet == "Fall"):
    desterm = fall 
elif(sublet == "Winter"):
    desterm = winter 
else: 
    desterm = summer 

filterdict = {} 


#Identification for the text, see more buttons, and title of each post 
class_name_text = "x1iorvi4 x1pi30zi x1l90r2v x1swvt13"
x_path_text = f".//div[@class='{class_name_text}']"

class_name_buttons = "x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1sur9pj xkrqix3 xzsf02u x1s688f"
x_path_more = f".//div[@class='{class_name_buttons}']"

class_name_title = "html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs"
x_path_title = f".//span[@class='{class_name_title}']"


#Go through each post in all_posts and attempt to find the text, link to post, and username of who posted 
for post in all_posts:
    try: 
        see_more_button = post.find_element(By.XPATH, x_path_more)
        actions.move_to_element(see_more_button).click().perform()
    except: 
        time.sleep(1) #continues on if no see more text exists 
        
    try:
        text_post = post.find_element(By.XPATH, x_path_text).text.lower()
        text_post = text_post.replace('\n','') #cleans the text when presented in dict
        for term in desterm:
            if term in text_post:
                link = post_link(post,actions,driver)
                title = post.find_element(By.XPATH,x_path_title).text
                filterdict[title] = [title,text_post, link]
                time.sleep(3)
                
    except NoSuchElementException:
        print("No Text Found, skipping ... ")
        
    except Exception as e:
        print(f"No text found for this post. Error: {e}")
    
        
            
    



19
No Text for post, skipping.
No Text for post, skipping.
No Text for post, skipping.
No Text for post, skipping.
No Text for post, skipping.
No Text for post, skipping.
No Text for post, skipping.
{'TK Chang': ['280 lester street (7th floor)1bedroom + 1den with door, fully furnished (with bed, desk, chair, sofa) 700sq feet condo unit for rent at $2300 a month (all utilities except electricity, wifi  included). 1 full + 1 half  washroom. central ac, heat, in-suite laundry provided.10minutes walk to uw and wlu. available from september 1,2024 to august 31,2025.', 'https://www.facebook.com/groups/1998166543836067/permalink/3752370645082306/?sale_post_id=3752370645082306'], 'Parthiv Patel': ['1 bed 1 bath - house667 karlsfeld rd, waterloo, on n2t 2v3, canada2/2 bedrooms basement unit  available for a 12 month lease starting september 2024 to august 25** both rooms available to sign lease as whole unit or you can sign a lease for separate rooms as well ***** see attached floor plan***** a

In [70]:
#Final Step: Display information in the form of dataframe 

new = pd.DataFrame.from_dict(filterdict)
pd.set_option('display.max_colwidth', None)
new.index = ['Text', 'Link']
new.columns = [f"Option {num}" for num in range(1, len(new.columns) + 1)]


display(new.style)

,Option 1,Option 2,Option 3
Text,"280 lester street (7th floor)1bedroom + 1den with door, fully furnished (with bed, desk, chair, sofa) 700sq feet condo unit for rent at $2300 a month (all utilities except electricity, wifi included). 1 full + 1 half washroom. central ac, heat, in-suite laundry provided.10minutes walk to uw and wlu. available from september 1,2024 to august 31,2025.","1 bed 1 bath - house667 karlsfeld rd, waterloo, on n2t 2v3, canada2/2 bedrooms basement unit available for a 12 month lease starting september 2024 to august 25** both rooms available to sign lease as whole unit or you can sign a lease for separate rooms as well ***** see attached floor plan***** address : 667 karlsfeld rd waterloo**• close to bus stops, plaza, uw & wlu• fully furnished with bed/desk/chair• separate and private side entrance from outside.• new washroom•• kitchen fully functional with hot plates• basement has laundry inside unit.** attached floor plan for the basemwnt unit* 850$ per room.* utilities are extra approximately 50$ *parking extra 50$feel free to each out if you’re interested or have any questions!","recently renovated house in foresthill, kitchener. the house has been upgraded to new building requirements, qualifying as a legal duplex. safety was #1 priority throughout the renovation. both units are equipped with interconnected fire and carbon monoxide alarms, fire dampers, hvr unit, thankless hot water and ac. this 3 bedroom with separate entrance is located in forest hill, kitchener. great family oriented neighborhood, walking distance to schools, parks, shops and restaurants. the apartment has full laminate flooring and pot lights throughout. kitchen is equipped with stainless steel fridge, stove, dishwasher. unit has its own washer and dryer. utilities & hydro are not included. unit also comes with roger's ignite 1gb internet service. upper unit has two parking space, one in the garage and the other on driveway. full access to a large backyard great for entertaining.note: this entire property (building and private grounds) is strictly non-smoking. this includes vaping and e-cigarettes. requirements: proof of employment, first two months’ rent, reference from previous landlord. full credit report will be required for all applicants. to book a viewing appointment or contact for further information:"
Link,https://www.facebook.com/groups/1998166543836067/permalink/3752370645082306/?sale_post_id=3752370645082306,https://www.facebook.com/groups/1998166543836067/permalink/3752339658418738/?sale_post_id=3752339658418738,https://www.facebook.com/groups/1998166543836067/permalink/3752264325092938/?sale_post_id=3752264325092938
